In [1]:
%matplotlib inline
from sklearn.neighbors import NearestNeighbors, KDTree

from mcmatch import cluster
from mcmatch.db.pg_database import PgFunDB
from mcmatch.util import extract_funname, signature_to_fname_heuristic
from mcmatch.feature.counter import counter_features
from mcmatch.feature import all_features
from mcmatch.feature.aggregator import FeatureAggregator
from sklearn import preprocessing
import pprint
import matplotlib.pyplot as plt
import timeit

metr_aggr = FeatureAggregator([all_features[m] for m in all_features])

fdb = PgFunDB()


In [2]:
def print_analysis(predictions, test_fun_fids, fid_names):
    good = 0
    nf = 0
    f = 0
    for i, v in enumerate(predictions):
        #if v == 'i2a' or v == 'dummy':
        #    continue
        fb = signature_to_fname_heuristic(test_fun_fids[i][1])
        if v == fb:
            good += 1
            print "**********", i, v, fb
        elif fb not in fid_names:
            nf += 1
            #print "!!", fb, "not in fid_names"
        #print i, v, fb
    
    print good, "of", len(predictions), float(good)/len(predictions)
    if nf != len(predictions):
        print good, "of", len(predictions)-nf, float(good)/(len(predictions)-nf)

In [3]:
def run_test_(fdb, metr_aggr, clf, trainset, transformMode=0):
    print "start"
    fids, train_data = fdb.get_features_np(metr_aggr, in_repositories=trainset)
    ppl = cluster.TransformPipeline(transformMode)
    train_data = ppl.transform_trainingset(train_data)
    fid_names = map(lambda z: signature_to_fname_heuristic(z[1]), fids)
    print "Fit"
    clf.fit(train_data, fid_names)
    del train_data
    test_fun_fids, test_fun_data = fdb.get_features_np(metr_aggr, in_repositories=['t-dietlibc'])
    test_fun_data = ppl.transform_testset(test_fun_data)
    predictions = clf.predict(test_fun_data)
    print_analysis(predictions, test_fun_fids, fid_names)

def run_test(fdb, metr_aggr, clf, trainset, transformMode=0):
    print timeit.timeit(lambda: run_test_(fdb, metr_aggr, clf, trainset, transformMode), number=1)

In [4]:
trainset1 = ['glibc', 'musl-1.1.6']
trainset2 = ['musl-1.1.6']
#trainset = ['glibc']

from sklearn import svm
from sklearn import tree
from sklearn import ensemble
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [5]:
run_test(fdb, metr_aggr, svm.SVC(), trainset1)

start
Fit
********** 78 ctime_r ctime_r
********** 150 getopt_long getopt_long
********** 342 atoi atoi
3 of 596 0.00503355704698
3 of 419 0.00715990453461
355.633279085


In [ ]:
run_test(fdb, metr_aggr, svm.SVC(), trainset2)

********** 187 atoi atoi
1 of 596 0.00167785234899
1 of 373 0.00268096514745


In [6]:
run_test(fdb, metr_aggr, svm.LinearSVC(), trainset1, 1)

start
Fit
********** 82 get_current_dir_name get_current_dir_name
********** 144 isalnum isalnum
********** 189 fwrite_unlocked fwrite_unlocked
********** 372 execlp execlp
********** 481 dn_skipname dn_skipname
********** 484 sigdelset sigdelset
********** 520 getpriority getpriority
********** 524 ftime ftime
********** 526 cfmakeraw cfmakeraw
********** 535 getopt getopt
********** 537 lockf lockf
********** 571 to64 to64
12 of 596 0.0201342281879
12 of 419 0.0286396181384
39788.8826408


In [5]:
run_test(fdb, metr_aggr, svm.LinearSVC(), trainset2)

start
Fit
********** 187 atoi atoi
********** 225 usleep usleep
********** 232 wcscpy wcscpy
********** 302 isalnum isalnum
********** 456 isalpha isalpha
********** 476 getservbyport getservbyport
********** 520 getpriority getpriority
********** 526 cfmakeraw cfmakeraw
8 of 596 0.0134228187919
8 of 373 0.0214477211796


In [6]:
run_test(fdb, metr_aggr, svm.LinearSVC(), trainset2, 1)

start
Fit
********** 33 siginterrupt siginterrupt
********** 81 execl execl
********** 82 get_current_dir_name get_current_dir_name
********** 225 usleep usleep
********** 302 isalnum isalnum
********** 481 dn_skipname dn_skipname
********** 520 getpriority getpriority
********** 524 ftime ftime
********** 535 getopt getopt
********** 537 lockf lockf
********** 571 to64 to64
11 of 596 0.0184563758389
11 of 373 0.029490616622


In [7]:
run_test(fdb, metr_aggr, svm.LinearSVC(), trainset1)

start
Fit
0 of 596 0.0
0 of 419 0.0
4752.93290114


In [7]:
run_test(fdb, metr_aggr, svm.SVC(), trainset1, 1)

start
Fit
********** 150 getopt_long getopt_long
********** 380 usleep usleep
********** 459 fdopendir fdopendir
********** 501 bsearch bsearch
4 of 596 0.00671140939597
4 of 419 0.00954653937947
306.691169977


In [ ]:
run_test(fdb, metr_aggr, svm.LinearSVC(), trainset2, 1)

In [12]:
run_test(fdb, metr_aggr, svm.LinearSVC(), trainset1, 1)

start
Fit
********** 82 get_current_dir_name get_current_dir_name
********** 144 isalnum isalnum
********** 200 atol atol
********** 372 execlp execlp
********** 481 dn_skipname dn_skipname
********** 520 getpriority getpriority
********** 524 ftime ftime
********** 526 cfmakeraw cfmakeraw
********** 535 getopt getopt
********** 537 lockf lockf
********** 571 to64 to64
11 of 596 0.0184563758389
11 of 419 0.0262529832936
8530.30809402


In [6]:
run_test(fdb, metr_aggr, svm.SVC(), trainset1, 2)

start
Fit
********** 78 ctime_r ctime_r
********** 187 atoi atoi
********** 308 getopt_long getopt_long
3 of 596 0.00503355704698
3 of 419 0.00715990453461


In [7]:
run_test(fdb, metr_aggr, svm.SVC(), trainset2, 2)

start
Fit
********** 187 atoi atoi
1 of 596 0.00167785234899
1 of 373 0.00268096514745


In [8]:
run_test(fdb, metr_aggr, tree.DecisionTreeClassifier(criterion="gini"), trainset2)

start
Fit
********** 2 isblank isblank
********** 58 memrchr memrchr
********** 81 execl execl
********** 82 get_current_dir_name get_current_dir_name
********** 152 iconv_open iconv_open
********** 185 flbf flbf
********** 187 atoi atoi
********** 206 mkfifo mkfifo
********** 302 isalnum isalnum
********** 319 md5crypt md5crypt
********** 355 abs abs
********** 379 execle execle
********** 411 isblank isblank
********** 456 isalpha isalpha
********** 526 cfmakeraw cfmakeraw
********** 550 wmemcpy wmemcpy
16 of 596 0.0268456375839
16 of 373 0.0428954423592


In [ ]:
run_test(fdb, metr_aggr, tree.DecisionTreeClassifier(criterion="gini"), trainset2, 1)

In [8]:
run_test(fdb, metr_aggr, tree.DecisionTreeClassifier(criterion="gini"), trainset1)

start
Fit
********** 2 isblank isblank
********** 59 strdup strdup
********** 144 isalnum isalnum
********** 155 dirname dirname
********** 197 abs abs
********** 200 atol atol
********** 221 execle execle
********** 253 isblank isblank
********** 340 flbf flbf
********** 361 mkfifo mkfifo
********** 456 isalpha isalpha
********** 490 fgetpos fgetpos
********** 501 bsearch bsearch
********** 520 getpriority getpriority
********** 526 cfmakeraw cfmakeraw
********** 537 lockf lockf
********** 549 wcswidth wcswidth
********** 550 wmemcpy wmemcpy
********** 571 to64 to64
19 of 596 0.0318791946309
19 of 419 0.0453460620525
28.862694025


In [9]:
run_test(fdb, metr_aggr, tree.DecisionTreeClassifier(criterion="gini"), trainset1, 1)

start
Fit
********** 2 isblank isblank
********** 144 isalnum isalnum
********** 155 dirname dirname
********** 197 abs abs
********** 200 atol atol
********** 221 execle execle
********** 253 isblank isblank
********** 340 flbf flbf
********** 361 mkfifo mkfifo
********** 456 isalpha isalpha
********** 490 fgetpos fgetpos
********** 501 bsearch bsearch
********** 520 getpriority getpriority
********** 526 cfmakeraw cfmakeraw
********** 537 lockf lockf
********** 550 wmemcpy wmemcpy
********** 571 to64 to64
17 of 596 0.0285234899329
17 of 419 0.0405727923628
26.9783201218


In [12]:
run_test(fdb, metr_aggr, tree.DecisionTreeClassifier(criterion="entropy"), trainset2)

start
Fit
********** 2 isblank isblank
********** 20 getprotobynumber getprotobynumber
********** 58 memrchr memrchr
********** 185 flbf flbf
********** 206 mkfifo mkfifo
********** 302 isalnum isalnum
********** 313 dirname dirname
********** 355 abs abs
********** 411 isblank isblank
********** 456 isalpha isalpha
********** 526 cfmakeraw cfmakeraw
********** 532 strptime strptime
********** 535 getopt getopt
********** 548 mbsinit mbsinit
14 of 596 0.0234899328859
14 of 373 0.0375335120643


In [13]:
run_test(fdb, metr_aggr, tree.DecisionTreeClassifier(criterion="entropy"), trainset2, 1)

start
Fit
********** 2 isblank isblank
********** 58 memrchr memrchr
********** 185 flbf flbf
********** 206 mkfifo mkfifo
********** 236 wmemset wmemset
********** 302 isalnum isalnum
********** 319 md5crypt md5crypt
********** 355 abs abs
********** 411 isblank isblank
********** 456 isalpha isalpha
********** 526 cfmakeraw cfmakeraw
********** 532 strptime strptime
********** 535 getopt getopt
********** 548 mbsinit mbsinit
********** 571 to64 to64
15 of 596 0.0251677852349
15 of 373 0.0402144772118


In [10]:
run_test(fdb, metr_aggr, tree.DecisionTreeClassifier(criterion="entropy"), trainset1)

start
Fit
********** 2 isblank isblank
********** 58 memrchr memrchr
********** 59 strdup strdup
********** 60 strndup strndup
********** 144 isalnum isalnum
********** 155 dirname dirname
********** 161 md5crypt md5crypt
********** 178 res_mkquery res_mkquery
********** 197 abs abs
********** 253 isblank isblank
********** 340 flbf flbf
********** 361 mkfifo mkfifo
********** 456 isalpha isalpha
********** 526 cfmakeraw cfmakeraw
********** 571 to64 to64
15 of 596 0.0251677852349
15 of 419 0.035799522673
57.2534070015


In [11]:
run_test(fdb, metr_aggr, tree.DecisionTreeClassifier(criterion="entropy"), trainset1, 1)

start
Fit
********** 2 isblank isblank
********** 20 getprotobynumber getprotobynumber
********** 58 memrchr memrchr
********** 59 strdup strdup
********** 144 isalnum isalnum
********** 155 dirname dirname
********** 161 md5crypt md5crypt
********** 178 res_mkquery res_mkquery
********** 197 abs abs
********** 213 getaddrinfo getaddrinfo
********** 253 isblank isblank
********** 340 flbf flbf
********** 361 mkfifo mkfifo
********** 456 isalpha isalpha
********** 526 cfmakeraw cfmakeraw
********** 532 strptime strptime
********** 571 to64 to64
17 of 596 0.0285234899329
17 of 419 0.0405727923628
51.8522019386


In [6]:
run_test(fdb, metr_aggr, RandomForestClassifier(), trainset1)

start
Fit
********** 2 isblank isblank
********** 31 sigaddset sigaddset
********** 59 strdup strdup
********** 70 adjtime adjtime
********** 81 execl execl
********** 144 isalnum isalnum
********** 155 dirname dirname
********** 178 res_mkquery res_mkquery
********** 197 abs abs
********** 253 isblank isblank
********** 307 iconv_open iconv_open
********** 456 isalpha isalpha
********** 484 sigdelset sigdelset
********** 501 bsearch bsearch
********** 520 getpriority getpriority
********** 526 cfmakeraw cfmakeraw
16 of 596 0.0268456375839
16 of 419 0.0381861575179
16.8397681713


In [19]:
run_test(fdb, metr_aggr, RandomForestClassifier(), trainset2)

start
Fit
********** 2 isblank isblank
********** 31 sigaddset sigaddset
********** 59 strdup strdup
********** 187 atoi atoi
********** 206 mkfifo mkfifo
********** 302 isalnum isalnum
********** 313 dirname dirname
********** 379 execle execle
********** 411 isblank isblank
********** 436 iconv iconv
********** 456 isalpha isalpha
********** 481 dn_skipname dn_skipname
********** 484 sigdelset sigdelset
********** 501 bsearch bsearch
********** 520 getpriority getpriority
********** 526 cfmakeraw cfmakeraw
********** 535 getopt getopt
********** 537 lockf lockf
********** 548 mbsinit mbsinit
19 of 596 0.0318791946309
19 of 373 0.0509383378016


In [11]:
run_test(fdb, metr_aggr, AdaBoostClassifier(n_estimators=100), trainset1)

start
Fit
0 of 596 0.0
0 of 419 0.0
359.227745056


In [ ]:

run_test(fdb, metr_aggr, GradientBoostingClassifier(), trainset1)

In [5]:
run_test(fdb, metr_aggr, BaggingClassifier(), trainset1)

start
Fit
********** 2 isblank isblank
********** 59 strdup strdup
********** 144 isalnum isalnum
********** 155 dirname dirname
********** 161 md5crypt md5crypt
********** 221 execle execle
********** 253 isblank isblank
********** 292 getgrouplist getgrouplist
********** 342 atoi atoi
********** 361 mkfifo mkfifo
********** 456 isalpha isalpha
********** 481 dn_skipname dn_skipname
********** 484 sigdelset sigdelset
********** 490 fgetpos fgetpos
********** 501 bsearch bsearch
********** 520 getpriority getpriority
********** 526 cfmakeraw cfmakeraw
********** 535 getopt getopt
********** 571 to64 to64
19 of 596 0.0318791946309
19 of 419 0.0453460620525
183.511810064


In [ ]:
run_test(fdb, metr_aggr, BaggingClassifier(), trainset2)

start
Fit
********** 2 isblank isblank
********** 59 strdup strdup
********** 152 iconv_open iconv_open
********** 187 atoi atoi
********** 206 mkfifo mkfifo
********** 277 if_nameindex if_nameindex
********** 302 isalnum isalnum
********** 313 dirname dirname
********** 319 md5crypt md5crypt
********** 359 labs labs
********** 377 ctime ctime
********** 379 execle execle
********** 411 isblank isblank
********** 456 isalpha isalpha
********** 501 bsearch bsearch
********** 526 cfmakeraw cfmakeraw
********** 537 lockf lockf
********** 549 wcswidth wcswidth
********** 571 to64 to64
19 of 596 0.0318791946309
19 of 373 0.0509383378016


In [ ]:
run_test(fdb, metr_aggr, ExtraTreesClassifier(), trainset1)

start
Fit

In [ ]:
run_test(fdb, metr_aggr, ExtraTreesClassifier(), trainset2)

In [ ]:
run_test(fdb, metr_aggr, GradientBoostingClassifier(), trainset1)

start
Fit

In [ ]:
run_test(fdb, metr_aggr, GradientBoostingClassifier(), trainset2)

In [8]:
run_test(fdb, metr_aggr, RandomForestClassifier(n_estimators=15), trainset1)

start
Fit
********** 2 isblank isblank
********** 13 mq_send mq_send
********** 31 sigaddset sigaddset
********** 59 strdup strdup
********** 70 adjtime adjtime
********** 71 cfsetospeed cfsetospeed
********** 81 execl execl
********** 144 isalnum isalnum
********** 155 dirname dirname
********** 219 ctime ctime
********** 253 isblank isblank
********** 307 iconv_open iconv_open
********** 340 flbf flbf
********** 342 atoi atoi
********** 361 mkfifo mkfifo
********** 456 isalpha isalpha
********** 484 sigdelset sigdelset
********** 501 bsearch bsearch
********** 520 getpriority getpriority
********** 526 cfmakeraw cfmakeraw
********** 535 getopt getopt
********** 537 lockf lockf
********** 549 wcswidth wcswidth
********** 571 to64 to64
24 of 596 0.0402684563758
24 of 419 0.0572792362768
23.8014822006


In [5]:
run_test(fdb, metr_aggr, RandomForestClassifier(n_estimators=15), trainset2)

start
Fit
********** 2 isblank isblank
********** 31 sigaddset sigaddset
********** 59 strdup strdup
********** 152 iconv_open iconv_open
********** 158 mq_receive mq_receive
********** 187 atoi atoi
********** 206 mkfifo mkfifo
********** 302 isalnum isalnum
********** 313 dirname dirname
********** 338 freadable freadable
********** 358 atol atol
********** 411 isblank isblank
********** 436 iconv iconv
********** 456 isalpha isalpha
********** 484 sigdelset sigdelset
********** 490 fgetpos fgetpos
********** 501 bsearch bsearch
********** 520 getpriority getpriority
********** 526 cfmakeraw cfmakeraw
********** 535 getopt getopt
********** 537 lockf lockf
********** 571 to64 to64
22 of 596 0.0369127516779
22 of 373 0.058981233244


In [6]:
run_test(fdb, metr_aggr, RandomForestClassifier(n_estimators=16), trainset2)

start
Fit
********** 2 isblank isblank
********** 13 mq_send mq_send
********** 31 sigaddset sigaddset
********** 59 strdup strdup
********** 152 iconv_open iconv_open
********** 187 atoi atoi
********** 206 mkfifo mkfifo
********** 302 isalnum isalnum
********** 313 dirname dirname
********** 319 md5crypt md5crypt
********** 336 res_mkquery res_mkquery
********** 411 isblank isblank
********** 456 isalpha isalpha
********** 484 sigdelset sigdelset
********** 490 fgetpos fgetpos
********** 501 bsearch bsearch
********** 519 bzero bzero
********** 520 getpriority getpriority
********** 526 cfmakeraw cfmakeraw
********** 535 getopt getopt
********** 537 lockf lockf
********** 548 mbsinit mbsinit
********** 549 wcswidth wcswidth
********** 550 wmemcpy wmemcpy
********** 571 to64 to64
25 of 596 0.0419463087248
25 of 373 0.0670241286863


In [7]:
run_test(fdb, metr_aggr, RandomForestClassifier(n_estimators=17), trainset2)

start
Fit
********** 2 isblank isblank
********** 31 sigaddset sigaddset
********** 59 strdup strdup
********** 81 execl execl
********** 144 opendir opendir
********** 152 iconv_open iconv_open
********** 173 sigemptyset sigemptyset
********** 187 atoi atoi
********** 206 mkfifo mkfifo
********** 217 execlp execlp
********** 302 isalnum isalnum
********** 313 dirname dirname
********** 338 freadable freadable
********** 359 labs labs
********** 379 execle execle
********** 411 isblank isblank
********** 436 iconv iconv
********** 456 isalpha isalpha
********** 484 sigdelset sigdelset
********** 501 bsearch bsearch
********** 526 cfmakeraw cfmakeraw
********** 535 getopt getopt
********** 549 wcswidth wcswidth
********** 550 wmemcpy wmemcpy
********** 571 to64 to64
25 of 596 0.0419463087248
25 of 373 0.0670241286863


In [8]:
run_test(fdb, metr_aggr, RandomForestClassifier(n_estimators=18), trainset2)

start
Fit
********** 2 isblank isblank
********** 31 sigaddset sigaddset
********** 59 strdup strdup
********** 81 execl execl
********** 152 iconv_open iconv_open
********** 206 mkfifo mkfifo
********** 213 gmtime gmtime
********** 302 isalnum isalnum
********** 313 dirname dirname
********** 349 putchar putchar
********** 358 atol atol
********** 411 isblank isblank
********** 436 iconv iconv
********** 456 isalpha isalpha
********** 481 dn_skipname dn_skipname
********** 484 sigdelset sigdelset
********** 490 fgetpos fgetpos
********** 501 bsearch bsearch
********** 520 getpriority getpriority
********** 526 cfmakeraw cfmakeraw
********** 535 getopt getopt
********** 537 lockf lockf
********** 548 mbsinit mbsinit
********** 549 wcswidth wcswidth
********** 550 wmemcpy wmemcpy
********** 571 to64 to64
26 of 596 0.0436241610738
26 of 373 0.0697050938338


In [9]:
run_test(fdb, metr_aggr, RandomForestClassifier(n_estimators=19), trainset2)

start
Fit
********** 2 isblank isblank
********** 31 sigaddset sigaddset
********** 59 strdup strdup
********** 82 get_current_dir_name get_current_dir_name
********** 152 iconv_open iconv_open
********** 206 mkfifo mkfifo
********** 302 isalnum isalnum
********** 313 dirname dirname
********** 359 labs labs
********** 379 execle execle
********** 411 isblank isblank
********** 456 isalpha isalpha
********** 481 dn_skipname dn_skipname
********** 484 sigdelset sigdelset
********** 501 bsearch bsearch
********** 526 cfmakeraw cfmakeraw
********** 535 getopt getopt
********** 537 lockf lockf
********** 548 mbsinit mbsinit
********** 549 wcswidth wcswidth
********** 571 to64 to64
21 of 596 0.0352348993289
21 of 373 0.0563002680965


In [10]:
run_test(fdb, metr_aggr, RandomForestClassifier(n_estimators=20), trainset2)

start
Fit
********** 2 isblank isblank
********** 31 sigaddset sigaddset
********** 59 strdup strdup
********** 152 iconv_open iconv_open
********** 180 fwritable fwritable
********** 206 mkfifo mkfifo
********** 217 execlp execlp
********** 302 isalnum isalnum
********** 313 dirname dirname
********** 319 md5crypt md5crypt
********** 336 res_mkquery res_mkquery
********** 355 abs abs
********** 358 atol atol
********** 379 execle execle
********** 411 isblank isblank
********** 436 iconv iconv
********** 456 isalpha isalpha
********** 484 sigdelset sigdelset
********** 501 bsearch bsearch
********** 526 cfmakeraw cfmakeraw
********** 537 lockf lockf
********** 571 to64 to64
22 of 596 0.0369127516779
22 of 373 0.058981233244


In [9]:
run_test(fdb, metr_aggr, BaggingClassifier(KNeighborsClassifier()), trainset1)

start
Fit
********** 200 atol atol
********** 201 labs labs
********** 219 ctime ctime
********** 456 isalpha isalpha
********** 532 strptime strptime
5 of 596 0.00838926174497
5 of 419 0.0119331742243
10.5274610519


In [13]:
run_test(fdb, metr_aggr, BaggingClassifier(KNeighborsClassifier()), trainset2)

start
Fit
********** 122 ldiv ldiv
********** 185 flbf flbf
********** 490 fgetpos fgetpos
********** 523 fstatvfs fstatvfs
4 of 596 0.00671140939597
4 of 373 0.0107238605898
